In [1]:
#데이터 가공 및 시각화를 위한 라이브러리 import
import pandas as pd
# matplotlib를 사용해 그래프를 그릴 때 사용할 글꼴을 설정하고 한글 깨짐을 수정함
from matplotlib import font_manager, rc
font_path = "C:/Windows/Fonts/NGULIM.TTF"
font = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font)
from datetime import datetime, timedelta
import os

C:\Users\User\anaconda3\envs\Air_pred\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\User\anaconda3\envs\Air_pred\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\User\anaconda3\envs\Air_pred\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
# 경고 메세지 off
import warnings

warnings.filterwarnings(action='ignore')

In [3]:
# 가공을 끝낸'df_pm10_drop' data set 불러오기
address = './Dataset'
df_pm10_drop = pd.read_csv(os.path.join(address,'df_pm10_drop_1110.csv'),encoding='cp949')

In [4]:
# 'df_pm10_drop'의 30일 기간의 시작시간과 끝 시간을 저장한 data set 불러오기
address = './Dataset/Dataset_info'
in_data_resample = pd.read_csv(os.path.join(address,'in_data_resample.csv'), header = 0, encoding='cp949')

In [5]:
in_data_resample.head()

,indata_cd,start_time,end_time
0,ICL1L2000011,2020-07-22,2020-08-21
1,ICL1L2000017,2020-06-27,2020-07-27
2,ICL1L2000064,2020-11-25,2020-12-25
3,ICL1L2000065,2020-12-18,2021-01-17
4,ICL1L2000234,2021-02-07,2021-03-09


In [6]:
# 길이가 부족한 indata_cd 삭제
index1 = in_data_resample[in_data_resample['start_time'] == '길이부족'].index
print(index1)
print(len(index1))

Int64Index([153], dtype='int64')
1


In [7]:
in_data_resample = in_data_resample.drop(index1)

In [8]:
# 결측값이 30%가 넘지 않는 30일 기간이 없는 indata_cd 삭제
index2 = in_data_resample[in_data_resample['start_time'] == 'false'].index
print(index2)
print(len(index2))

Int64Index([145], dtype='int64')
1


In [9]:
in_data_resample = in_data_resample.drop(index2)

In [10]:
# in_data_resample의 값 datetime으로 type 변경
in_data_resample['start_time_2'] =pd.to_datetime(in_data_resample['start_time'],format='%Y-%m-%d',errors='raise')

In [11]:
# 30일 데이터에서 7일 기간 계산
in_data_resample['start_time_2']= in_data_resample['start_time_2'].map(lambda x : x )
in_data_resample['end_time_2']= in_data_resample['start_time_2'].map(lambda x : x + timedelta(6))
in_data_resample['end_time_2'] = in_data_resample['end_time_2'].copy().dt.date
in_data_resample['start_time_2'] = in_data_resample['start_time_2'].copy().dt.date
in_data_resample['start_time_2'] = in_data_resample['start_time_2'].astype(str)
in_data_resample['end_time_2']  = in_data_resample['end_time_2'].astype(str)

In [12]:
in_data_resample.head()
# start_time_2:7일 데이터 시작 , end_time_2: 7일 데이터 끝

,indata_cd,start_time,end_time,start_time_2,end_time_2
0,ICL1L2000011,2020-07-22,2020-08-21,2020-07-22,2020-07-28
1,ICL1L2000017,2020-06-27,2020-07-27,2020-06-27,2020-07-03
2,ICL1L2000064,2020-11-25,2020-12-25,2020-11-25,2020-12-01
3,ICL1L2000065,2020-12-18,2021-01-17,2020-12-18,2020-12-24
4,ICL1L2000234,2021-02-07,2021-03-09,2021-02-07,2021-02-13


In [13]:
# 계산된 7일 기간 dataframe 저장
address = './Data'
in_data_resample.to_csv(os.path.join(address,'in_data_resample_7day.csv'), encoding="cp949", index=False)

In [14]:
# 계산된 7일 기간으로 'df_pm10_drop'를 filtering 진행

df_pm10_7day_result = pd.DataFrame()
for idx, row in in_data_resample.iterrows():
    # print(row)
    if row['start_time'] == 'false':
        pass

    elif row['start_time'] == '길이부족':
        pass

    else:
        print(row['indata_cd'], row['start_time_2'], row['end_time_2'])
        sample = df_pm10_drop[df_pm10_drop['in_id'] == row['indata_cd']]
        in_serial = sample.iloc[1,0]

        filtered_df =sample.loc[sample["time_day"].between(row['start_time_2'], row['end_time_2'])] #7일 기간 filtering

        filtered_df['time_m']= filtered_df['time'].apply(lambda _ : datetime.strptime(_,'%Y-%m-%d %H:%M:%S'))
        #filtered_df['time_day']= filtered_df['time_day'].apply(lambda _ : datetime.strptime(_,'%Y-%m-%d'))
        filtered_df = filtered_df.set_index(keys='time_m')
        filtered_df_result = filtered_df.resample(rule='10T').mean() # 10분 단위로 resampling
        print(len(filtered_df_result)) # 데이터 길이 출력
        if len(filtered_df_result) < 1008 :
            print(row['indata_cd'])

        dict = {'in_serial': in_serial,'in_id': row['indata_cd'],'time_day':filtered_df_result.index,'in_pm10_day': filtered_df_result['in_pm10'].values} # 가공된 in_id를 dict에 저장
        update= pd.DataFrame(dict)
        df_pm10_7day_result = pd.concat([df_pm10_7day_result,update], ignore_index=True)

ICL1L2000011 2020-07-22 2020-07-28
1008
ICL1L2000017 2020-06-27 2020-07-03
1008
ICL1L2000064 2020-11-25 2020-12-01
1008
ICL1L2000065 2020-12-18 2020-12-24
1008
ICL1L2000234 2021-02-07 2021-02-13
1008
ICL1L2000235 2021-02-07 2021-02-13
1008
ICL1L2000236 2021-02-06 2021-02-12
1008
ICL1L2000237 2021-02-06 2021-02-12
1008
ICL1L2000238 2021-03-14 2021-03-20
1008
ICL1L2000239 2021-02-07 2021-02-13
1008
ICL1L2000240 2021-02-07 2021-02-13
1008
ICL1L2000241 2021-02-07 2021-02-13
1008
ICL1L2000242 2021-02-07 2021-02-13
1008
ICL1L2000244 2021-02-07 2021-02-13
1008
ICL1L2000245 2021-02-07 2021-02-13
1008
ICL1L2000246 2021-02-07 2021-02-13
1008
ICL1L2000247 2021-02-07 2021-02-13
1008
ICL1L2000248 2021-02-07 2021-02-13
1008
ICL1L2000249 2021-02-07 2021-02-13
1008
ICL1L2000252 2021-02-07 2021-02-13
1008
ICL1L2000253 2021-02-07 2021-02-13
1008
ICL1L2000254 2021-02-07 2021-02-13
1008
ICL1L2000255 2021-02-07 2021-02-13
1008
ICL1L2000257 2021-03-24 2021-03-30
1008
ICL1L2000258 2021-02-07 2021-02-13
1008


In [15]:
df_pm10_7day_result

,in_serial,in_id,time_day,in_pm10_day
0,요양원,ICL1L2000011,2020-07-22 00:00:00,21.5
1,요양원,ICL1L2000011,2020-07-22 00:10:00,22.3
2,요양원,ICL1L2000011,2020-07-22 00:20:00,21.8
3,요양원,ICL1L2000011,2020-07-22 00:30:00,21.5
4,요양원,ICL1L2000011,2020-07-22 00:40:00,21.5
...,...,...,...,...
153206,요양원,IS70W2000858,2021-04-29 23:10:00,11.1
153207,요양원,IS70W2000858,2021-04-29 23:20:00,10.6
153208,요양원,IS70W2000858,2021-04-29 23:30:00,11.1
153209,요양원,IS70W2000858,2021-04-29 23:40:00,9.8


In [16]:
# 7일 기간 데이터 csv로 저장
address = './Dataset'
df_pm10_7day_result.to_csv(os.path.join(address,'df_pm10_7day_result_1110.csv'), encoding="cp949", index=False)